In [1]:
#1.
import numpy as np

# 參數
n = 200  # 數據點數量
mu = 0.01
phi1 = 0.1
phi2 = 0
phi3 = -0.1
sigma = 0.1  # 噪音的標準差

# 生成AR(3)時間序列
np.random.seed(0)  # 確保可重現性
R = np.zeros(n)
for t in range(3, n):
    R[t] = mu + phi1 * R[t-1] + phi2 * R[t-2] + phi3 * R[t-3] + np.random.normal(scale=sigma)

# 打印前200個數據點
print(R[:200])

[ 0.          0.          0.          0.18640523  0.06865624  0.11473942
  0.22692274  0.21258245 -0.07794349  0.07452222 -0.01894174  0.00557829
  0.04416546  0.03071508  0.15794103  0.09748133  0.02884413  0.04147663
  0.03776696  0.16030019  0.00136653  0.03766673 -0.08767292 -0.25420293
  0.04617489  0.1098284  -0.02781337  0.22957664 -0.12346174  0.00501101
 -0.03117495  0.1725066   0.17368544  0.04598078  0.03516367 -0.09263075
 -0.2019408  -0.04850166  0.03004781  0.15622793  0.15071094 -0.01666637
 -0.0375197  -0.11367836 -0.14170299 -0.17104535  0.19934084 -0.00686083
 -0.01738898 -0.13695252  0.07473987 -0.1421769  -0.01179647 -0.08820029
  0.05408791 -0.03449208 -0.1026924  -0.00849625  0.05543277  0.03246424
  0.04434324 -0.05454116 -0.03497466 -0.06517783 -0.02701898 -0.07051906
 -0.16316238  0.01412827 -0.02171336 -0.13887493  0.0409779  -0.07446071
  0.02163596  0.08097486  0.03844185  0.12562066 -0.109018    0.03548818
 -0.06749426 -0.07292734 -0.05872652 -0.02027848  0

In [20]:
#2.
n = 200  # 数据点数量
burn_in = 1000  # 燃入期迭代次数
iterations = 5000  # Gibbs采样迭代次数
missing_index = 100  # 缺失观测值的索引

# 观测数据（不包括 t=100 的数据）
# 假设 R 是观察到的数据
# R = ...

# 复制观测数据来处理缺失值
R_missing = np.copy(R)
R_missing[missing_index] = np.nan  # 将缺失值设置为 NaN

# 初始化参数
mu = 0.01
phi1 = 0.1
phi2 = 0
phi3 = -0.1
sigma = 0.1

# Gibbs采样
for i in range(burn_in + iterations):
    # 步骤1：采样 R[100]
    if i >= burn_in:
        R_missing[missing_index] = np.random.normal(mu + phi1 * R_missing[missing_index - 1] + phi2 * R_missing[missing_index - 3], sigma)

    # 步骤2：采样参数
    # 更新 mu
    mu_mean = np.nanmean(R_missing - phi1 * np.roll(R_missing, 1) - phi2 * np.roll(R_missing, 2) - phi3 * np.roll(R_missing, 3))
    mu_std = np.nanstd(R_missing - phi1 * np.roll(R_missing, 1) - phi2 * np.roll(R_missing, 2) - phi3 * np.roll(R_missing, 3))
    mu = np.random.normal(mu_mean, mu_std)

    # 更新 phi1、phi2、phi3
    # 类似地，使用它们各自的条件分布更新其他参数（phi1、phi2、phi3、sigma）

# 打印出缺失值的估计结果
print("缺失值的估计结果 (t=100):", R_missing[missing_index])

缺失值的估计结果 (t=100): -0.1808558014908239


In [21]:
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
# 尝试拟合不同的模型来确定数据的模型
# 以下是一个简单的示例使用 AutoReg 拟合 AR 模型
best_aic = float('inf')
best_model = None

for lag in range(1, 5):
    model = AutoReg(R_missing, lags=lag).fit()
    aic = model.aic

    print(f"滞后项数量为 {lag} 时的模型 AIC 值：{aic}")

    if aic < best_aic:
        best_aic = aic
        best_model = model

# 打印最佳模型摘要
print("\n最佳模型的摘要：")
print(best_model.summary())

滞后项数量为 1 时的模型 AIC 值：-341.99304846497097
滞后项数量为 2 时的模型 AIC 值：-337.96102263027547
滞后项数量为 3 时的模型 AIC 值：-334.7430255381703
滞后项数量为 4 时的模型 AIC 值：-334.6783796534682

最佳模型的摘要：
                            AutoReg Model Results                             
Dep. Variable:                      y   No. Observations:                  200
Model:                     AutoReg(1)   Log Likelihood                 173.997
Method:               Conditional MLE   S.D. of innovations              0.101
Date:                Fri, 24 May 2024   AIC                           -341.993
Time:                        01:48:47   BIC                           -332.113
Sample:                             1   HQIC                          -337.994
                                  200                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0115      0.007      1.60

In [17]:
#3.
from scipy.stats import norm

# 遺失值區間的索引範圍
missing_indices = np.arange(100, 111)

# 初始化觀察資料和遺失值
observed_data = np.copy(R)
missing_data = observed_data[missing_indices]
observed_data[missing_indices] = np.nan

# 初始化參數
mu = 0.01
phi1 = 0.1
phi2 = 0
phi3 = -0.1
sigma = 0.1

# Gibbs采樣
for i in range(burn_in + iterations):
    # 步驟1：采樣遺失值
    for index in missing_indices:
        observed_data[index] = np.random.normal(mu + phi1 * observed_data[index - 1] + phi2 * observed_data[index - 3], sigma)

    # 步驟2：采樣參數
    mu_mean = np.nanmean(observed_data - phi1 * np.roll(observed_data, 1) - phi2 * np.roll(observed_data, 2) - phi3 * np.roll(observed_data, 3))
    mu_std = np.nanstd(observed_data - phi1 * np.roll(observed_data, 1) - phi2 * np.roll(observed_data, 2) - phi3 * np.roll(observed_data, 3))
    mu = np.random.normal(mu_mean, mu_std)

    # 更新 phi1、phi2、phi3、sigma
    # ...

# 打印填補後的遺失值
print("填補後的遺失值 (t=100~110):", observed_data[missing_indices])

填補後的遺失值 (t=100~110): [ 0.07714291 -0.0407787   0.06781355 -0.0349634   0.02299548  0.13294874
 -0.03985276 -0.04764528 -0.05450399 -0.01419598  0.15996974]


In [18]:
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA

# 初始化不同模型的AIC值
best_aic = np.inf
best_model = None
best_model_type = None

# 遍歷可能的模型
for lag in range(1, 6):  # 尝试不同的滞后项数量
    # 尝试AutoReg模型
    ar_model = AutoReg(observed_data, lags=lag).fit()
    ar_aic = ar_model.aic

    # 尝试ARIMA模型
    arima_model = ARIMA(observed_data, order=(lag, 0, 0)).fit()
    arima_aic = arima_model.aic

    # 比较AIC值
    if ar_aic < best_aic:
        best_aic = ar_aic
        best_model = ar_model
        best_model_type = "AutoReg"

    if arima_aic < best_aic:
        best_aic = arima_aic
        best_model = arima_model
        best_model_type = "ARIMA"

# 打印最佳模型的AIC值和摘要
print(f"最佳模型类型: {best_model_type}")
print(f"最佳模型的AIC值: {best_aic}")
print(best_model.summary())

最佳模型类型: ARIMA
最佳模型的AIC值: -357.29631491552027
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  200
Model:                 ARIMA(1, 0, 0)   Log Likelihood                 181.648
Date:                Fri, 24 May 2024   AIC                           -357.296
Time:                        01:36:10   BIC                           -347.401
Sample:                             0   HQIC                          -353.292
                                - 200                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0144      0.008      1.744      0.081      -0.002       0.030
ar.L1          0.1574      0.068      2.318      0.020       0.024       0.291
sigma2 